### First, import the necessary libraries

In [89]:
from sklearn.cluster import KMeans
import numpy as np
import seaborn as sns
import csv
import os
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import Dense, Dropout

In [90]:
# import os
# os.environ["CUDA_VISIBLE_DEVICES"]="1"


In [91]:
from sklearn.preprocessing import MinMaxScaler

X_train = pd.read_csv('UNSW-NB15/a part of training and testing set/UNSW_NB15_testing-set.csv', header=0) # they flipped the names.
X_test = pd.read_csv('UNSW-NB15/a part of training and testing set/UNSW_NB15_training-set.csv', header=0)

X_train = X_train.dropna()
X_test=  X_test.dropna()

Separate labels from data

In [92]:
y_train = X_train.iloc[:,-1]
X_train = X_train.iloc[:,:9]

y_test = X_test.iloc[:,-1]
X_test = X_test.iloc[:,:9]

In [93]:
# categorical
for col in ['proto', 'service', 'state']:
    X_train[col] = pd.Categorical(X_train[col]).codes
    X_test[col] = pd.Categorical(X_test[col]).codes

X_train.pop('id')
X_test.pop('id')

0            1
1            2
2            3
3            4
4            5
         ...  
82327    82328
82328    82329
82329    82330
82330    82331
82331    82332
Name: id, Length: 82332, dtype: int64

In [94]:
# Remove columns created with "algorithms in C#" because they never say how they got them... not repeatable


## Scale the data (BIG, BIG PERFORMANCE INCREASE!!!)

In [95]:
scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

## what if I just threw a NN at it, or "How I became a data scientist in one easy step"

In [96]:
# define the keras model
batch_size = len(X_train)

model = tf.keras.Sequential()
model.add(Dense(64, input_dim=8, activation='tanh'))
model.add(Dense(64, activation='tanh'))
model.add(Dense(64, activation='tanh'))
model.add(Dense(64, activation='tanh'))
model.add(Dense(32, activation='tanh'))
model.add(Dense(32, activation='tanh'))
model.add(Dense(32, activation='tanh'))
model.add(Dense(16, activation='tanh'))
model.add(Dense(16, activation='tanh'))
model.add(Dense(16, activation='tanh'))
model.add(Dense(16, activation='tanh'))
model.add(Dense(8, activation='tanh'))
model.add(Dense(8, activation='tanh'))
model.add(Dense(1, activation='sigmoid'))

In [97]:
from tensorflow.keras.optimizers import Adam
optim = Adam(learning_rate=0.003)

redlr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=10, min_lr=0.00001, verbose=True)
earlystop = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=30, restore_best_weights=True, verbose=True)
model.compile(loss = tf.keras.losses.BinaryCrossentropy(), optimizer = optim, metrics=['accuracy'])

In [98]:
#shuffle train myself, tf doesnt do that when i use val split

shuffler = np.random.permutation(len(X_train))
X_train = X_train[shuffler]
y_train = y_train[shuffler]

test_dataset = tf.data.Dataset.from_tensor_slices((X_test, y_test.values))
test_dataset = test_dataset.shuffle(len(X_test)).batch(batch_size)

In [99]:
history = model.fit(X_train, y_train.values, epochs=2000, batch_size=batch_size, validation_split=0.15, callbacks=[redlr, earlystop] )

Epoch 1/2000
1/1 [==============================] - 0s 473ms/step - loss: 0.7447 - accuracy: 0.3185 - val_loss: 0.6429 - val_accuracy: 0.6757
Epoch 2/2000
1/1 [==============================] - 0s 29ms/step - loss: 0.6406 - accuracy: 0.6815 - val_loss: 0.6305 - val_accuracy: 0.6757
Epoch 3/2000
1/1 [==============================] - 0s 24ms/step - loss: 0.6272 - accuracy: 0.6815 - val_loss: 0.6280 - val_accuracy: 0.6757
Epoch 4/2000
1/1 [==============================] - 0s 27ms/step - loss: 0.6241 - accuracy: 0.6815 - val_loss: 0.6279 - val_accuracy: 0.6757
Epoch 5/2000
1/1 [==============================] - 0s 27ms/step - loss: 0.6234 - accuracy: 0.6815 - val_loss: 0.6286 - val_accuracy: 0.6757
Epoch 6/2000
1/1 [==============================] - 0s 25ms/step - loss: 0.6238 - accuracy: 0.6815 - val_loss: 0.6278 - val_accura

In [100]:
model.save('less_features_fanning_layers_1024max_higher_starting_lr_no_dropout.h5')

In [101]:
y_pred = model.predict(X_test)

In [102]:
precision = tf.keras.metrics.Precision()
precision.update_state(y_test, y_pred)
precision.result().numpy()

0.5832054

In [103]:
recall = tf.keras.metrics.Recall()
recall.update_state(y_test, y_pred)
recall.result().numpy()

0.9972205

In [104]:
accuracy = tf.keras.metrics.BinaryAccuracy()
accuracy.update_state(y_test, y_pred)
accuracy.result().numpy()

0.6060705

In [105]:
actual_prediction = np.round(y_pred)

# actually normal, predicted normal
normal_traffic_indices = np.argwhere(y_test.to_numpy()==0)
normal_traffic_indices = normal_traffic_indices.reshape((len(normal_traffic_indices),))
attack_traffic_indices = np.argwhere(y_test.to_numpy()==1)
attack_traffic_indices = attack_traffic_indices.reshape((len(attack_traffic_indices),))

In [106]:
tn = len(np.where(np.take(actual_prediction, normal_traffic_indices, axis=0)==0)[0])
norm = len(normal_traffic_indices)
print(f'classified {tn} out of {norm} normal traffic patterns as normal {tn/norm}%')

classified 4693 out of 37000 normal traffic patterns as normal 0.12683783783783784%


In [107]:
fn = len(np.where(np.take(actual_prediction, attack_traffic_indices, axis=0)==0)[0])
attack = len(attack_traffic_indices)
print(f'classified {fn} out of {attack} attack traffic patterns as normal {fn/attack}%')

classified 126 out of 45332 attack traffic patterns as normal 0.002779493514515133%


In [108]:
fp = len(np.where(np.take(actual_prediction, normal_traffic_indices, axis=0)==1)[0])
print(f'classified {fp} out of {norm} normal traffic patterns as attack {fp/norm}%')

classified 32307 out of 37000 normal traffic patterns as attack 0.8731621621621621%


In [109]:
tp = len(np.where(np.take(actual_prediction, attack_traffic_indices, axis=0)==1)[0])
print(f'classified {tp} out of {attack} attack traffic patterns as attack {tp/attack}%')

classified 45206 out of 45332 attack traffic patterns as attack 0.9972205064854849%


In [110]:
tf.keras.utils.plot_model(model, to_file='model_plot.png', show_shapes=True, show_layer_names=False)

('You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) ', 'for plot_model/model_to_dot to work.')
